## <span style="color:purple">ArcGIS API for Python: Traffic and Pedestrian Activity Detection</span>

![detection](../img/DCActivityDetection.gif "CityWideDetection")

## Integrating ArcGIS with TensorFlow Deep Learning using the ArcGIS API for Python

## Washington, DC Camera Network Activity Capturing

This notebook provides an example of integration between ArcGIS and deep learning frameworks like TensorFlow using the ArcGIS API for Python.

<img src="../img/ArcGIS_ML_Integration.png" style="width: 75%"></img>

We will leverage a model to detect objects on a live video feed from youtube, and use these to update a feature service on a web GIS in real-time. As people, cars, trucks, and buses are detected on the feed, the feature will be updated to reflect the detection. 

# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import getpass
import shutil

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from PIL import ImageGrab
import time
import pandas as pd

import cv2

import requests
from io import BytesIO

from copy import deepcopy

import scipy.misc

from datetime import datetime

import arcgis

## Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Establish Connection to ArcGIS Online via ArcGIS API for Python

#### Authenticate

In [ ]:
gis_url = ""   # Replace with gis URL
username = ""  # Replace with username
gis = arcgis.gis.GIS(gis_url, username)

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

### Model: SSD Mobilenet on COCO

In [ ]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

This is a helper function that takes the detection graph output tensor (np arrays), stacks the classes and scores, and determines if the class for a person (1) is available within a certain score and within a certain amount of objects

In [ ]:
def object_counter(classes_arr, scores_arr, score_thresh=0.3):
    # Process the numpy array of classes from the model
    stacked_arr = np.stack((classes_arr, scores_arr), axis=-1)
    # Convert to pandas dataframe for easier querying
    detection_df = pd.DataFrame(stacked_arr)
    # Retrieve total count of cars with score threshold above param value
    detected_cars = detection_df[(detection_df[0] == 3.0) & (detection_df[1] > score_thresh)]
    detected_people =  detection_df[(detection_df[0] == 1.0) & (detection_df[1] > score_thresh)]
    detected_bicycles =  detection_df[(detection_df[0] == 2.0) & (detection_df[1] > score_thresh)]
    detected_motorcycles =  detection_df[(detection_df[0] == 4.0) & (detection_df[1] > score_thresh)]
    detected_buses =  detection_df[(detection_df[0] == 6.0) & (detection_df[1] > score_thresh)]
    detected_trucks =  detection_df[(detection_df[0] == 8.0) & (detection_df[1] > score_thresh)]
    
    car_count = len(detected_cars)
    people_count = len(detected_people)
    bicycle_count = len(detected_bicycles)
    motorcycle_count = len(detected_motorcycles)
    bus_count = len(detected_buses)
    truck_count = len(detected_trucks)

    return car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count

def return_lat(row):
    return row['location']['latitude']

def return_lon(row):
    return row['location']['longitude']

# Network Camera Processing

In [ ]:
# TrafficLand API source JSON
trafficland_API_key = os.environ['TRAFFICLAND_API_KEY']  # 
source_json = r"http://api.trafficland.com/v1.5/json/video_feeds?system=ddot&key={0}".format(trafficland_API_key)
df = pd.read_json(source_json)
ddot_df = df.loc[df.provider == 'DDOT']
# Edit the ddot_df to make a feature service using the API
ddot_df['latitude'] = ddot_df.apply(lambda row: return_lat(row), axis=1)
ddot_df['longitude'] = ddot_df.apply(lambda row: return_lon(row), axis=1)

# Multiple Camera Iteration Prototyping

In [ ]:
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
jpeg_size = "halfJpeg" # Other Options: halfJpeg; fullJpeg; hugeJpeg

## P1: Use existing feature service and send model outputs as updates

RT Service

In [ ]:
# object_point_srvc = gis.content.search("DDOT Traffic Cameras", item_type="feature service")[1]
# object_point_srvc

Stable Service

In [ ]:
object_point_srvc = gis.content.search("DDOT_TrafficCameras_CityNetwork", item_type="feature service")[0]
object_point_srvc

In [ ]:
# Convert our existing service into a pandas dataframe
object_point_lyr = object_point_srvc.layers[0]
obj_fset = object_point_lyr.query()  #querying without any conditions returns all the features
obj_df = obj_fset.df
all_features = obj_fset.features
original_feature = all_features[1]
feature_to_be_updated = deepcopy(original_feature)

## Model Update with Attachments and Trends

In [ ]:
overwrite_attachment = True

upload_source = False

write_trends = True

reset_trends = True

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for iteration in range(1000):
            # TrafficLand API source JSON
            trafficland_API_key = os.environ['TRAFFICLAND_API_KEY']  # 
            source_json = r"http://api.trafficland.com/v1.5/json/video_feeds?system=ddot&key={0}".format(trafficland_API_key)
            df = pd.read_json(source_json)
            ddot_df = df.loc[df.provider == 'DDOT']
            
            for ix in range(1,110):
                print("Processing camera {0} of 110...".format(str(ix)))

                # Retrieve camera information
                camera_jpeg_url = ddot_df.iloc[ix]['content']['hugeJpeg']
                camera_name = ddot_df.iloc[ix]['name']
                camera_id = ddot_df.iloc[ix]['publicId']
                
                # Retrieve the camera image
                camera_response = requests.get(camera_jpeg_url)
                
#                 # Perform attachment of source image
                if upload_source:
                    streamed_response = requests.get(camera_jpeg_url, stream=True)
                    with open('source.jpg', 'wb') as out_file:
                        shutil.copyfileobj(streamed_response.raw, out_file)
    #                 del response                

                image = Image.open(BytesIO(camera_response.content))
                image_np = load_image_into_numpy_array(image)
                image_np_expanded = np.expand_dims(image_np, axis=0)

                (boxes, scores, classes, num) = sess.run(
                      [detection_boxes, detection_scores, detection_classes, num_detections],
                      feed_dict={image_tensor: image_np_expanded})

                vis_util.visualize_boxes_and_labels_on_image_array(
                      image_np,
                      np.squeeze(boxes),
                      np.squeeze(classes).astype(np.int32),
                      np.squeeze(scores),
                      category_index,
                      use_normalized_coordinates=True,
                      line_thickness=8,
                      min_score_thresh=0.3
                )
                
                car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count = object_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
                vehicle_count = car_count + motorcycle_count + bus_count + truck_count
                total_count = vehicle_count + bicycle_count + people_count
                
                # Retrieve the feature from the feature layer to update
                obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
                all_features = obj_fset.features
                original_feature = all_features[0]
                feature_to_be_updated = deepcopy(original_feature)
                
                features_for_update = []
                feature_to_be_updated.attributes['rt_object_count'] = total_count
                feature_to_be_updated.attributes['rt_vehicle_count'] = vehicle_count
                feature_to_be_updated.attributes['rt_car_count'] = car_count
                feature_to_be_updated.attributes['rt_bus_count'] = bus_count
                feature_to_be_updated.attributes['rt_truck_count'] = truck_count
                feature_to_be_updated.attributes['rt_motorcycle_count'] = motorcycle_count
                feature_to_be_updated.attributes['rt_pedestrian_count'] = people_count
                feature_to_be_updated.attributes['rt_bicycle_count'] = bicycle_count
                
                feature_to_be_updated.attributes['source_image'] = camera_jpeg_url
                
                # Perform Trend Process
                if write_trends:
                    
                    # If the trend needs to be reset
                    if reset_trends and iteration==0:
                        print("Resetting trends...")
                        feature_to_be_updated.attributes['histval_10am_th_car'] = car_count
                        feature_to_be_updated.attributes['histval_10am_th_ped'] = people_count
                        feature_to_be_updated.attributes['histval_10am_th_bus'] = bus_count
                        feature_to_be_updated.attributes['histval_10am_th_trck'] = truck_count
                        
                        histval_car = car_count
                        histval_ped = people_count
                        histval_bus = bus_count
                        histval_truck = truck_count

                    # If the trend does not need to be reset
                    else:
                        histval_car = feature_to_be_updated.attributes['histval_10am_th_car']
                        histval_ped = feature_to_be_updated.attributes['histval_10am_th_ped']
                        histval_bus = feature_to_be_updated.attributes['histval_10am_th_bus']
                        histval_truck = feature_to_be_updated.attributes['histval_10am_th_trck']
                        
                        if histval_car == None:
                            histval_car = car_count
                        if histval_bus == None:
                            histval_bus = bus_count
                        if histval_truck == None:
                            histval_truck = truck_count
                        if histval_ped == None:
                            histval_ped = people_count                            
                    
                        # Calculate the new trend value and store it for the new update
                        feature_to_be_updated.attributes['histval_10am_th_car'] = (car_count + histval_car) / 2
                        feature_to_be_updated.attributes['histval_10am_th_ped'] = (people_count + histval_ped) / 2
                        feature_to_be_updated.attributes['histval_10am_th_bus'] = (bus_count + histval_bus) / 2
                        feature_to_be_updated.attributes['histval_10am_th_trck'] = (truck_count + histval_truck) / 2
                        
                    # Determine threshold percent, value, and status
                    thrshpcnt_car = feature_to_be_updated.attributes['threshold_val_cars']
                    thrshpcnt_ped = feature_to_be_updated.attributes['threshold_val_ped']
                    thrshpcnt_bus = feature_to_be_updated.attributes['threshold_val_buses']
                    thrshpcnt_truck = feature_to_be_updated.attributes['threshold_val_trck']
                    
                    # Perform writes to the threshold status field
                    
                    if car_count > (histval_car * thrshpcnt_car):
                        feature_to_be_updated.attributes['thrsh_status_cars'] = "Above"           
                    else:
                        feature_to_be_updated.attributes['thrsh_status_cars'] = "Below"
                        
                    if people_count > (histval_ped * thrshpcnt_ped):
                        feature_to_be_updated.attributes['thrsh_status_ped'] = "Above"
                    else:
                        feature_to_be_updated.attributes['thrsh_status_ped'] = "Below"                        
                        
                    if bus_count > (histval_bus * thrshpcnt_bus):
                        feature_to_be_updated.attributes['thrsh_status_bus'] = "Above"
                    else:
                        feature_to_be_updated.attributes['thrsh_status_bus'] = "Below"                        
                        
                    if truck_count > (histval_truck * thrshpcnt_truck):
                        feature_to_be_updated.attributes['thrsh_status_trck'] = "Above"
                    else:
                        
                        feature_to_be_updated.attributes['thrsh_status_trck'] = "Below"                        
                                               
                # Store attribute updates and send edit request
                features_for_update.append(feature_to_be_updated)
                object_point_lyr.edit_features(updates=features_for_update)    
        
                # Perform attachment of detected image
                scipy.misc.imsave('detected_objs.jpg', image_np)
                obj_id = feature_to_be_updated.attributes['F__OBJECTID']
                
                if overwrite_attachment:
                    for attachment in object_point_lyr.attachments.get_list(obj_id):
                        object_point_lyr.attachments.delete(obj_id, attachment['id'])
                                               
                object_point_lyr.attachments.add(obj_id, 'detected_objs.jpg')
                
                if upload_source:
                    object_point_lyr.attachments.add(obj_id, 'source.jpg')
                                    

## Reset Routine (WARNING: Will erase trends in data!)

https://esrifederal.maps.arcgis.com/home/webmap/viewer.html?webmap=255196da558749209dc63c07ffb38ae6

In [ ]:
for iteration in range(1):
    
    features_for_update = []
            
    for ix in range(0,111):
        print("Resetting values for camera {0} of 100...".format(str(ix)))

        # Retrieve camera information
        camera_name = ddot_df.iloc[ix]['name']

        # Retrieve the feature from the feature layer to update
        obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
        all_features = obj_fset.features
        original_feature = all_features[0]
        feature_to_be_updated = deepcopy(original_feature)

        
        feature_to_be_updated.attributes['rt_object_count'] = 0
        feature_to_be_updated.attributes['rt_vehicle_count'] = 0
        feature_to_be_updated.attributes['rt_car_count'] = 0
        feature_to_be_updated.attributes['rt_bus_count'] = 0
        feature_to_be_updated.attributes['rt_truck_count'] = 0
        feature_to_be_updated.attributes['rt_motorcycle_count'] = 0
        feature_to_be_updated.attributes['rt_pedestrian_count'] = 0
        feature_to_be_updated.attributes['rt_bicycle_count'] = 0
        
        feature_to_be_updated.attributes['histval_10am_th_car'] = None
        feature_to_be_updated.attributes['histval_10am_th_ped'] = None
        feature_to_be_updated.attributes['histval_10am_th_bus'] = None
        feature_to_be_updated.attributes['histval_10am_th_trck'] = None
        
        feature_to_be_updated.attributes['thrsh_status_cars'] = "Not Tracked"
        feature_to_be_updated.attributes['thrsh_status_ped'] = "Not Tracked" 
        feature_to_be_updated.attributes['thrsh_status_bus'] = "Not Tracked" 
        feature_to_be_updated.attributes['thrsh_status_trck'] = "Not Tracked"           

        features_for_update.append(feature_to_be_updated)
        
    object_point_lyr.edit_features(updates=features_for_update)

Set trend threshold percentages

In [ ]:
thresh_percent = 2.0

reset_trends = True

for iteration in range(1):
    
    features_for_update = []
            
    for ix in range(0,111):
        print("Setting trend threshold values for camera {0} of 111...".format(str(ix)))

        # Retrieve camera information
        camera_name = ddot_df.iloc[ix]['name']

        # Retrieve the feature from the feature layer to update
        obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
        all_features = obj_fset.features
        original_feature = all_features[0]
        feature_to_be_updated = deepcopy(original_feature)
        
        feature_to_be_updated.attributes['threshold_val_cars'] = thresh_percent
        feature_to_be_updated.attributes['threshold_val_ped'] = thresh_percent
        feature_to_be_updated.attributes['threshold_val_buses'] = thresh_percent
        feature_to_be_updated.attributes['threshold_val_trck'] = thresh_percent    
        
        feature_to_be_updated.attributes['thrsh_status_cars'] = "Not Tracked"
        feature_to_be_updated.attributes['thrsh_status_ped'] = "Not Tracked" 
        feature_to_be_updated.attributes['thrsh_status_bus'] = "Not Tracked" 
        feature_to_be_updated.attributes['thrsh_status_trck'] = "Not Tracked" 
        
        if reset_trends:
            feature_to_be_updated.attributes['histval_10am_th_car'] = None
            feature_to_be_updated.attributes['histval_10am_th_ped'] = None
            feature_to_be_updated.attributes['histval_10am_th_bus'] = None
            feature_to_be_updated.attributes['histval_10am_th_trck'] = None


        features_for_update.append(feature_to_be_updated)
        
    object_point_lyr.edit_features(updates=features_for_update)

### Perform Demo Updates (Model, Attachments, Trends) Against Constitution Ave Event Features

Set thresh percent

In [ ]:
event_feature_names = [
    "9th St @ Constitution Ave",
    "12th St @ Constitution Ave",
    "14th St @ Pennsylvania Ave",
    "Pennsylvania Ave @ 7th St",
    "Pennsylvania Ave @ 9th St"
]

thresh_percent = 1.125

features_for_update = []

for event_feature_name in event_feature_names:
    print("Setting thresh percent values for camera '{0}'...".format(str(event_feature_name)))

    record = ddot_df.loc[ddot_df.name == event_feature_name]

    # Retrieve camera information
    camera_name = record['name'].all()

    # Retrieve the feature from the feature layer to update
    obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
    all_features = obj_fset.features
    original_feature = all_features[0]
    feature_to_be_updated = deepcopy(original_feature)

    feature_to_be_updated.attributes['threshold_val_cars'] = thresh_percent
    feature_to_be_updated.attributes['threshold_val_ped'] = thresh_percent
    feature_to_be_updated.attributes['threshold_val_buses'] = thresh_percent
    feature_to_be_updated.attributes['threshold_val_trck'] = thresh_percent
    
    features_for_update.append(feature_to_be_updated)

object_point_lyr.edit_features(updates=features_for_update)   

Reset Trend Values for Event Locations

In [ ]:
event_feature_names = [
    "9th St @ Constitution Ave",
    "12th St @ Constitution Ave",
    "14th St @ Pennsylvania Ave",
    "Pennsylvania Ave @ 7th St",
    "Pennsylvania Ave @ 9th St"
]

features_for_update = []

for event_feature_name in event_feature_names:
    print("Resetting values for camera '{0}'...".format(str(event_feature_name)))

    record = ddot_df.loc[ddot_df.name == event_feature_name]

    # Retrieve camera information
    camera_name = record['name'].all()

    # Retrieve the feature from the feature layer to update
    obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
    all_features = obj_fset.features
    original_feature = all_features[0]
    feature_to_be_updated = deepcopy(original_feature)


    feature_to_be_updated.attributes['rt_object_count'] = 0
    feature_to_be_updated.attributes['rt_vehicle_count'] = 0
    feature_to_be_updated.attributes['rt_car_count'] = 0
    feature_to_be_updated.attributes['rt_bus_count'] = 0
    feature_to_be_updated.attributes['rt_truck_count'] = 0
    feature_to_be_updated.attributes['rt_motorcycle_count'] = 0
    feature_to_be_updated.attributes['rt_pedestrian_count'] = 0
    feature_to_be_updated.attributes['rt_bicycle_count'] = 0

    feature_to_be_updated.attributes['histval_10am_th_car'] = None
    feature_to_be_updated.attributes['histval_10am_th_ped'] = None
    feature_to_be_updated.attributes['histval_10am_th_bus'] = None
    feature_to_be_updated.attributes['histval_10am_th_trck'] = None
    
    feature_to_be_updated.attributes['histval_10am_th_trck'] = None
    
    feature_to_be_updated.attributes['thrsh_status_cars'] = "Not Tracked"
    feature_to_be_updated.attributes['thrsh_status_ped'] = "Not Tracked" 
    feature_to_be_updated.attributes['thrsh_status_bus'] = "Not Tracked" 
    feature_to_be_updated.attributes['thrsh_status_trck'] = "Not Tracked"      

    features_for_update.append(feature_to_be_updated)

object_point_lyr.edit_features(updates=features_for_update)   

##### Demo Event Features Against Screencap

1920 x 1080

Single Feature

In [ ]:
event_feature_name = "9th St @ Constitution Ave"

overwrite_attachment = True

write_trends = True

reset_trends = True

iteration=0

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        while True: 
            
            record = ddot_df.loc[ddot_df.name == event_feature_name]

            # Retrieve camera information
            camera_name = record['name'].all()

            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np = np.array(ImageGrab.grab(bbox=(0,510,960,1020)))
            image_np_expanded = np.expand_dims(image_np, axis=0)

            (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
            
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8, min_score_thresh=0.5)
            
            cv2.imshow('object detection', cv2.resize(image_np, (960,510), interpolation=cv2.INTER_CUBIC))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break            

            car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count = object_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
            vehicle_count = car_count + motorcycle_count + bus_count + truck_count
            total_count = vehicle_count + bicycle_count + people_count

            # Retrieve the feature from the feature layer to update
            obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
            all_features = obj_fset.features
            original_feature = all_features[0]
            feature_to_be_updated = deepcopy(original_feature)

            features_for_update = []
            feature_to_be_updated.attributes['rt_object_count'] = total_count
            feature_to_be_updated.attributes['rt_vehicle_count'] = vehicle_count
            feature_to_be_updated.attributes['rt_car_count'] = car_count
            feature_to_be_updated.attributes['rt_bus_count'] = bus_count
            feature_to_be_updated.attributes['rt_truck_count'] = truck_count
            feature_to_be_updated.attributes['rt_motorcycle_count'] = motorcycle_count
            feature_to_be_updated.attributes['rt_pedestrian_count'] = people_count
            feature_to_be_updated.attributes['rt_bicycle_count'] = bicycle_count

            # Perform Trend Process
            if write_trends:

                # If the trend needs to be reset
                if reset_trends and iteration==0:
                    print("Resetting trends...")
                    feature_to_be_updated.attributes['histval_10am_th_car'] = car_count
                    feature_to_be_updated.attributes['histval_10am_th_ped'] = people_count
                    feature_to_be_updated.attributes['histval_10am_th_bus'] = bus_count
                    feature_to_be_updated.attributes['histval_10am_th_trck'] = truck_count

                    histval_car = car_count
                    histval_ped = people_count
                    histval_bus = bus_count
                    histval_truck = truck_count

                # If the trend does not need to be reset
                else:
                    histval_car = feature_to_be_updated.attributes['histval_10am_th_car']
                    histval_ped = feature_to_be_updated.attributes['histval_10am_th_ped']
                    histval_bus = feature_to_be_updated.attributes['histval_10am_th_bus']
                    histval_truck = feature_to_be_updated.attributes['histval_10am_th_trck']

                    if histval_car == None:
                        histval_car = car_count
                    if histval_ped == None:
                        histval_ped = people_count                           
                    if histval_bus == None:
                        histval_bus = bus_count
                    if histval_truck == None:
                        histval_truck = truck_count
                         
                    # Calculate the new trend value and store it for the new update
                    feature_to_be_updated.attributes['histval_10am_th_car'] = (car_count + histval_car) / 2
                    feature_to_be_updated.attributes['histval_10am_th_ped'] = (people_count + histval_ped) / 2
                    feature_to_be_updated.attributes['histval_10am_th_bus'] = (bus_count + histval_bus) / 2
                    feature_to_be_updated.attributes['histval_10am_th_trck'] = (truck_count + histval_truck) / 2

                # Determine threshold percent, value, and status
                thrshpcnt_car = feature_to_be_updated.attributes['threshold_val_cars']
                thrshpcnt_ped = feature_to_be_updated.attributes['threshold_val_ped']
                thrshpcnt_bus = feature_to_be_updated.attributes['threshold_val_buses']
                thrshpcnt_truck = feature_to_be_updated.attributes['threshold_val_trck']

                # Perform writes to the threshold status field

                if car_count > (histval_car * thrshpcnt_car):
                    feature_to_be_updated.attributes['thrsh_status_cars'] = "Above"           
                else:
                    feature_to_be_updated.attributes['thrsh_status_cars'] = "Below"

                if people_count > (histval_ped * thrshpcnt_ped):
                    feature_to_be_updated.attributes['thrsh_status_ped'] = "Above"
                else:
                    feature_to_be_updated.attributes['thrsh_status_ped'] = "Below"                        

                if bus_count > (histval_bus * thrshpcnt_bus):
                    feature_to_be_updated.attributes['thrsh_status_bus'] = "Above"
                else:
                    feature_to_be_updated.attributes['thrsh_status_bus'] = "Below"                        

                if truck_count > (histval_truck * thrshpcnt_truck):
                    feature_to_be_updated.attributes['thrsh_status_trck'] = "Above"
                    "Passed truck thresh"
                else:
                    feature_to_be_updated.attributes['thrsh_status_trck'] = "Below"                        

            # Store attribute updates and send edit request
            features_for_update.append(feature_to_be_updated)
            object_point_lyr.edit_features(updates=features_for_update)    

            # Perform attachment of detected image
            scipy.misc.imsave('detected_objs.jpg', image_np)
            obj_id = feature_to_be_updated.attributes['F__OBJECTID']

            if overwrite_attachment:
                for attachment in object_point_lyr.attachments.get_list(obj_id):
                    object_point_lyr.attachments.delete(obj_id, attachment['id'])

            object_point_lyr.attachments.add(obj_id, 'detected_objs.jpg')
               
            iteration+=1

Multiple Features

In [ ]:
event_feature_names = [
    "9th St @ Constitution Ave",
    "12th St @ Constitution Ave",
    "14th St @ Pennsylvania Ave",
    "Pennsylvania Ave @ 7th St",
    "Pennsylvania Ave @ 9th St"
]

overwrite_attachment = True

write_trends = True

reset_trends = True

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        while True:
            
            for event_feature_name in event_feature_names:
                print("Processing camera '{0}' (Iteration {1})...".format(str(camera_name), str(iteration)))
                record = ddot_df.loc[ddot_df.name == event_feature_name]

                # Retrieve camera information
                camera_name = record['name'].all()

                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np = np.array(ImageGrab.grab(bbox=(0,510,960,1020)))
                image_np_expanded = np.expand_dims(image_np, axis=0)

                (boxes, scores, classes, num) = sess.run(
                      [detection_boxes, detection_scores, detection_classes, num_detections],
                      feed_dict={image_tensor: image_np_expanded})

                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  line_thickness=8, min_score_thresh=0.5)

                cv2.imshow('object detection', cv2.resize(image_np, (960,510), interpolation=cv2.INTER_CUBIC))
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break            

                car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count = object_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
                vehicle_count = car_count + motorcycle_count + bus_count + truck_count
                total_count = vehicle_count + bicycle_count + people_count

                # Retrieve the feature from the feature layer to update
                obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
                all_features = obj_fset.features
                original_feature = all_features[0]
                feature_to_be_updated = deepcopy(original_feature)

                features_for_update = []
                feature_to_be_updated.attributes['rt_object_count'] = total_count
                feature_to_be_updated.attributes['rt_vehicle_count'] = vehicle_count
                feature_to_be_updated.attributes['rt_car_count'] = car_count
                feature_to_be_updated.attributes['rt_bus_count'] = bus_count
                feature_to_be_updated.attributes['rt_truck_count'] = truck_count
                feature_to_be_updated.attributes['rt_motorcycle_count'] = motorcycle_count
                feature_to_be_updated.attributes['rt_pedestrian_count'] = people_count
                feature_to_be_updated.attributes['rt_bicycle_count'] = bicycle_count

                feature_to_be_updated.attributes['source_image'] = camera_jpeg_url

                # Perform Trend Process
                if write_trends:

                    # If the trend needs to be reset
                    if reset_trends and iteration==0:
                        print("Resetting trends...")
                        feature_to_be_updated.attributes['histval_10am_th_car'] = car_count
                        feature_to_be_updated.attributes['histval_10am_th_ped'] = people_count
                        feature_to_be_updated.attributes['histval_10am_th_bus'] = bus_count
                        feature_to_be_updated.attributes['histval_10am_th_trck'] = truck_count

                        histval_car = car_count
                        histval_ped = people_count
                        histval_bus = bus_count
                        histval_truck = truck_count

                    # If the trend does not need to be reset
                    else:
                        histval_car = feature_to_be_updated.attributes['histval_10am_th_car']
                        histval_ped = feature_to_be_updated.attributes['histval_10am_th_ped']
                        histval_bus = feature_to_be_updated.attributes['histval_10am_th_bus']
                        histval_truck = feature_to_be_updated.attributes['histval_10am_th_trck']

                        if histval_car == None:
                            histval_car = car_count
                        if histval_ped == None:
                            histval_ped = people_count                           
                        if histval_bus == None:
                            histval_bus = bus_count
                        if histval_truck == None:
                            histval_truck = truck_count

                        # Calculate the new trend value and store it for the new update
                        feature_to_be_updated.attributes['histval_10am_th_car'] = (car_count + histval_car) / 2
                        feature_to_be_updated.attributes['histval_10am_th_ped'] = (people_count + histval_ped) / 2
                        feature_to_be_updated.attributes['histval_10am_th_bus'] = (bus_count + histval_bus) / 2
                        feature_to_be_updated.attributes['histval_10am_th_trck'] = (truck_count + histval_truck) / 2

                    # Determine threshold percent, value, and status
                    thrshpcnt_car = feature_to_be_updated.attributes['threshold_val_cars']
                    thrshpcnt_ped = feature_to_be_updated.attributes['threshold_val_ped']
                    thrshpcnt_bus = feature_to_be_updated.attributes['threshold_val_buses']
                    thrshpcnt_truck = feature_to_be_updated.attributes['threshold_val_trck']

                    # Perform writes to the threshold status field

                    if car_count > (histval_car * thrshpcnt_car):
                        feature_to_be_updated.attributes['thrsh_status_cars'] = "Above"           
                    else:
                        feature_to_be_updated.attributes['thrsh_status_cars'] = "Below"

                    if people_count > (histval_ped * thrshpcnt_ped):
                        feature_to_be_updated.attributes['thrsh_status_ped'] = "Above"
                    else:
                        feature_to_be_updated.attributes['thrsh_status_ped'] = "Below"                        

                    if bus_count > (histval_bus * thrshpcnt_bus):
                        feature_to_be_updated.attributes['thrsh_status_bus'] = "Above"
                    else:
                        feature_to_be_updated.attributes['thrsh_status_bus'] = "Below"                        

                    if truck_count > (histval_truck * thrshpcnt_truck):
                        feature_to_be_updated.attributes['thrsh_status_trck'] = "Above"
                    else:
                        feature_to_be_updated.attributes['thrsh_status_trck'] = "Below"                        

                # Store attribute updates and send edit request
                features_for_update.append(feature_to_be_updated)
                object_point_lyr.edit_features(updates=features_for_update)    

                # Perform attachment of detected image
                scipy.misc.imsave('detected_objs.jpg', image_np)
                obj_id = feature_to_be_updated.attributes['F__OBJECTID']

                if overwrite_attachment:
                    for attachment in object_point_lyr.attachments.get_list(obj_id):
                        object_point_lyr.attachments.delete(obj_id, attachment['id'])

                object_point_lyr.attachments.add(obj_id, 'detected_objs.jpg')

                if upload_source:
                    object_point_lyr.attachments.add(obj_id, 'source.jpg')

##### Demo Event Features Against Traffic Cameras

In [ ]:
demo_event_features = True

event_feature_names = [
    "9th St @ Constitution Ave",
    "12th St @ Constitution Ave"
]

overwrite_attachment = True

upload_source = False

write_trends = True

reset_trends = True

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for iteration in range(1, 21):
            # TrafficLand API source JSON
            source_json = r"http://api.trafficland.com/v1.5/json/video_feeds?system=ddot&key=1594c8892d7fbd18181a8a6a44958b28"
            df = pd.read_json(source_json)
            ddot_df = df.loc[df.provider == 'DDOT']
            
            if demo_event_features:
                for event_feature_name in event_feature_names:
                    print("Processing camera '{0}' (Iteration {1})...".format(str(camera_name), str(iteration)))
                    
                    record = ddot_df.loc[ddot_df.name == event_feature_name]
                    
                    # Retrieve camera information
                    camera_jpeg_url = record['content'].all()['hugeJpeg']
                    camera_name = record['name'].all()
                    camera_id = record['publicId'].all()

                    # Retrieve the camera image
                    camera_response = requests.get(camera_jpeg_url)

    #                 # Perform attachment of source image
                    if upload_source:
                        streamed_response = requests.get(camera_jpeg_url, stream=True)
                        with open('source.jpg', 'wb') as out_file:
                            shutil.copyfileobj(streamed_response.raw, out_file)
        #                 del response                

                    image = Image.open(BytesIO(camera_response.content))
                    image_np = load_image_into_numpy_array(image)
                    image_np_expanded = np.expand_dims(image_np, axis=0)

                    (boxes, scores, classes, num) = sess.run(
                          [detection_boxes, detection_scores, detection_classes, num_detections],
                          feed_dict={image_tensor: image_np_expanded})

                    vis_util.visualize_boxes_and_labels_on_image_array(
                          image_np,
                          np.squeeze(boxes),
                          np.squeeze(classes).astype(np.int32),
                          np.squeeze(scores),
                          category_index,
                          use_normalized_coordinates=True,
                          line_thickness=8)

                    car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count = object_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
                    vehicle_count = car_count + motorcycle_count + bus_count + truck_count
                    total_count = vehicle_count + bicycle_count + people_count

                    # Retrieve the feature from the feature layer to update
                    obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""", return_geometry=False)  
                    all_features = obj_fset.features
                    original_feature = all_features[0]
                    feature_to_be_updated = deepcopy(original_feature)

                    features_for_update = []
                    feature_to_be_updated.attributes['rt_object_count'] = total_count
                    feature_to_be_updated.attributes['rt_vehicle_count'] = vehicle_count
                    feature_to_be_updated.attributes['rt_car_count'] = car_count
                    feature_to_be_updated.attributes['rt_bus_count'] = bus_count
                    feature_to_be_updated.attributes['rt_truck_count'] = truck_count
                    feature_to_be_updated.attributes['rt_motorcycle_count'] = motorcycle_count
                    feature_to_be_updated.attributes['rt_pedestrian_count'] = people_count
                    feature_to_be_updated.attributes['rt_bicycle_count'] = bicycle_count

                    feature_to_be_updated.attributes['source_image'] = camera_jpeg_url

                    # Perform Trend Process
                    if write_trends:

                        # If the trend needs to be reset
                        if reset_trends and iteration==0:
                            print("Resetting trends...")
                            feature_to_be_updated.attributes['histval_10am_th_car'] = car_count
                            feature_to_be_updated.attributes['histval_10am_th_ped'] = people_count
                            feature_to_be_updated.attributes['histval_10am_th_bus'] = bus_count
                            feature_to_be_updated.attributes['histval_10am_th_trck'] = truck_count

                            histval_car = car_count
                            histval_ped = people_count
                            histval_bus = bus_count
                            histval_truck = truck_count

                        # If the trend does not need to be reset
                        else:
                            histval_car = feature_to_be_updated.attributes['histval_10am_th_car']
                            histval_ped = feature_to_be_updated.attributes['histval_10am_th_ped']
                            histval_bus = feature_to_be_updated.attributes['histval_10am_th_bus']
                            histval_truck = feature_to_be_updated.attributes['histval_10am_th_trck']

                            if histval_car == None:
                                histval_car = car_count
                            if histval_bus == None:
                                histval_bus = bus_count
                            if histval_truck == None:
                                histval_truck = truck_count
                            if histval_ped == None:
                                histval_ped = people_count                            

                            # Calculate the new trend value and store it for the new update
                            feature_to_be_updated.attributes['histval_10am_th_car'] = (car_count + histval_car) / 2
                            feature_to_be_updated.attributes['histval_10am_th_ped'] = (people_count + histval_ped) / 2
                            feature_to_be_updated.attributes['histval_10am_th_bus'] = (bus_count + histval_bus) / 2
                            feature_to_be_updated.attributes['histval_10am_th_trck'] = (truck_count + histval_truck) / 2

                        # Determine threshold percent, value, and status
                        thrshpcnt_car = feature_to_be_updated.attributes['threshold_val_cars']
                        thrshpcnt_ped = feature_to_be_updated.attributes['threshold_val_ped']
                        thrshpcnt_bus = feature_to_be_updated.attributes['threshold_val_buses']
                        thrshpcnt_truck = feature_to_be_updated.attributes['threshold_val_trck']

                        # Perform writes to the threshold status field

                        if car_count > (histval_car * thrshpcnt_car):
                            feature_to_be_updated.attributes['thrsh_status_cars'] = "Above"           
                        else:
                            feature_to_be_updated.attributes['thrsh_status_cars'] = "Below"

                        if people_count > (histval_ped * thrshpcnt_ped):
                            feature_to_be_updated.attributes['thrsh_status_ped'] = "Above"
                        else:
                            feature_to_be_updated.attributes['thrsh_status_ped'] = "Below"                        

                        if bus_count > (histval_bus * thrshpcnt_bus):
                            feature_to_be_updated.attributes['thrsh_status_bus'] = "Above"
                        else:
                            feature_to_be_updated.attributes['thrsh_status_bus'] = "Below"                        

                        if truck_count > (histval_truck * thrshpcnt_truck):
                            feature_to_be_updated.attributes['thrsh_status_trck'] = "Above"
                        else:
                            feature_to_be_updated.attributes['thrsh_status_trck'] = "Below"                        

                    # Store attribute updates and send edit request
                    features_for_update.append(feature_to_be_updated)
                    object_point_lyr.edit_features(updates=features_for_update)    

                    # Perform attachment of detected image
                    scipy.misc.imsave('detected_objs.jpg', image_np)
                    obj_id = feature_to_be_updated.attributes['F__OBJECTID']

                    if overwrite_attachment:
                        for attachment in object_point_lyr.attachments.get_list(obj_id):
                            object_point_lyr.attachments.delete(obj_id, attachment['id'])

                    object_point_lyr.attachments.add(obj_id, 'detected_objs.jpg')

                    if upload_source:
                        object_point_lyr.attachments.add(obj_id, 'source.jpg')

## P2: Perform Updates Along Constitution Avenue

Perform run on subset along Constitution Ave.

In [ ]:
object_point_srvc_const_ave = gis.content.search("DC_TrafficCameras_ConstitutionAve", item_type="feature service")[0]
object_point_srvc_const_ave

In [ ]:
# Convert our existing service into a pandas dataframe
object_point_lyr_const_ave = object_point_srvc_const_ave.layers[0]
obj_fset_const_ave = object_point_lyr_const_ave.query()  #querying without any conditions returns all the features
obj_df_const_ave = obj_fset_const_ave.df
constitutionave_names_list = obj_df_const_ave.name.tolist()
ddot_constitutionave_df = ddot_df[ddot_df.name.isin(constitutionave_names_list)]

https://esrifederal.maps.arcgis.com/home/webmap/viewer.html?webmap=e081b389ac8d4bdb817c34dbe78a5fc4

## P3: Trend-setter

In [ ]:
object_point_srvc = gis.content.search("DC_TrafficCameras_HIST_ConstAve", item_type="feature service")[0]
object_point_srvc

In [ ]:
# Convert our existing service into a pandas dataframe
object_point_lyr = object_point_srvc.layers[0]
obj_fset = object_point_lyr.query()  #querying without any conditions returns all the features
obj_df = obj_fset.df
obj_df.head()

With auto-refresh of source JSON

In [ ]:
with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
        
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for refresh_iteration in range(10):
            print("Refresh iteration {0}...".format(str(refresh_iteration)))
            
            # TrafficLand API source JSON
            source_json = r"http://api.trafficland.com/v1.5/json/video_feeds?system=ddot&key=1594c8892d7fbd18181a8a6a44958b28"
            df = pd.read_json(source_json)
            ddot_df = df.loc[df.provider == 'DDOT']
            ddot_constitutionave_df = ddot_df[ddot_df.name.isin(constitutionave_names_list)]
            
            for iteration in range(100):
                print("Iteration: {0} of {1}".format(str(iteration), str(100)))

                for ix, row in enumerate(ddot_constitutionave_df.iterrows()):

                    # Retrieve camera information
                    camera_jpeg_url = ddot_constitutionave_df.iloc[ix]['content']['hugeJpeg']
                    camera_name = ddot_constitutionave_df.iloc[ix]['name']
                    camera_id = ddot_constitutionave_df.iloc[ix]['publicId']

                    # Retrieve the camera image
                    camera_response = requests.get(camera_jpeg_url)

                    image = Image.open(BytesIO(camera_response.content))
                    image_np = load_image_into_numpy_array(image)
                    image_np_expanded = np.expand_dims(image_np, axis=0)

                    (boxes, scores, classes, num) = sess.run(
                          [detection_boxes, detection_scores, detection_classes, num_detections],
                          feed_dict={image_tensor: image_np_expanded})

                    vis_util.visualize_boxes_and_labels_on_image_array(
                          image_np,
                          np.squeeze(boxes),
                          np.squeeze(classes).astype(np.int32),
                          np.squeeze(scores),
                          category_index,
                          use_normalized_coordinates=True,
                          line_thickness=8)

                    car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count = object_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
                    vehicle_count = car_count + motorcycle_count + bus_count + truck_count
                    total_count = vehicle_count + bicycle_count + people_count

                    # Retrieve the feature from the feature layer to update
                    obj_fset = object_point_lyr.query(where="""name = '"""+camera_name+"""'""")  
                    all_features = obj_fset.features
                    original_feature = all_features[0]

                    features_to_be_added = []
                    new_feature = deepcopy(original_feature)

                    new_feature.attributes['rt_object_count'] = total_count
                    new_feature.attributes['rt_vehicle_count'] = vehicle_count
                    new_feature.attributes['rt_car_count'] = car_count
                    new_feature.attributes['rt_bus_count'] = bus_count
                    new_feature.attributes['rt_truck_count'] = truck_count
                    new_feature.attributes['rt_motorcycle_count'] = motorcycle_count
                    new_feature.attributes['rt_pedestrian_count'] = people_count
                    new_feature.attributes['rt_bicycle_count'] = bicycle_count
                    new_feature.attributes['recording_id'] = "Cam{0}_{1}".format(str(ix),
                                                                                 str(datetime.now().strftime("%m%d%y_%H%M%S")))
                    new_feature.attributes['observation_time'] = datetime.now().strftime("%m/%d/%y %H:%M:%S")

                    #add this to the list of features to be updated
                    features_to_be_added.append(new_feature)

                    object_point_lyr.edit_features(adds = features_to_be_added) 